In [1]:
# None 값이 있는지 확인
import pandas as pd
import json

file_path = "../data/scraping/all_book_data_ver3.json"
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# JSON 데이터가 리스트인지 딕셔너리인지 확인 후 변환
if isinstance(data, list):
    df = pd.DataFrame(data)
elif isinstance(data, dict):  # 딕셔너리 형태이면 변환 (예: {key: list})
    df = pd.DataFrame.from_dict(data)

# None 값 확인
none_count = df.isna().sum()

print(none_count)

title                0
description          0
key_sentences        0
author               0
publisher            0
isbn13              13
img_url              0
publication_date    14
dtype: int64


중복 null 13개 + pulication_date 1개 => 총 14개 null

In [3]:
import pandas as pd
import json

# JSON 파일 로드
file_path = "../data/scraping/all_book_data_ver3.json"  # 파일 경로 설정
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# JSON 데이터를 DataFrame으로 변환
if isinstance(data, list):
    df = pd.DataFrame(data)
elif isinstance(data, dict):  # 딕셔너리 형태이면 변환
    df = pd.DataFrame.from_dict(data)

# 📌 'ISBN' 접두사 제거
df["isbn13"] = df["isbn13"].astype(str).str.replace(r'^ISBN', '', regex=True)

# 📌 이미지 URL 변환 (JSON 데이터가 로드된 후 직접 수정)
df["img_url"] = df["img_url"].astype(str).apply(lambda x: x.replace("\\/", "/"))

# 📌 `isbn13` 값이 숫자가 아닌 경우 삭제
df = df[df["isbn13"].str.isdigit()]  # 숫자로만 이루어진 isbn13 값만 유지

# 📌 'isbn13' 또는 'publication_date' 값이 None인 행 삭제
df.dropna(subset=['isbn13', 'publication_date'], inplace=True)

# 📌 모든 컬럼에서 하나라도 None 값이 있는 행 삭제
df.dropna(inplace=True)

# 📌 수정된 데이터를 JSON으로 저장
output_path = "../data/scraping/all_book_data_ver3_cleaned.json"
df.to_json(output_path, orient="records", force_ascii=False, indent=4)

print(f"✅ 수정 완료! 저장된 파일: {output_path}")

FileNotFoundError: [Errno 2] No such file or directory: '../data/scraping/all_book_data_ver3[0_20000].json'

#### 목적 : 데이터베이스가 테이블이 생성되고 json 파일이랑 DB랑 연결되는지 테스트

In [ ]:
# Python 경로 추가
import sys
import os
import json
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))

from app.database.connection import DatabaseSettings, create_connection
from app.services.data_ingestion_service import create_tables
from app.services.data_insert_DB import insert_books_and_authors

# 데이터베이스 연결 설정
db_settings = DatabaseSettings()
conn = create_connection(db_settings)

if conn:
    # 테이블 생성
    create_tables(conn)
    print('success')

    # JSON 파일 경로
    json_file_path = "../data/scraping/all_book_data_ver3_cleaned.json"

    # JSON 데이터를 테이블에 삽입
    insert_books_and_authors(conn, json_file_path)

    # 데이터 조회 테스트
    def test_connection(conn):
        """테이블 데이터 조회 테스트"""
        try:
            cursor = conn.cursor()
            cursor.execute("SELECT * FROM books;")
            rows = cursor.fetchall()
            if rows:
                for row in rows:
                    print(row)
            else:
                print("No data found in 'books' table.")
        except Exception as e:
            print("Error during connection test:", e)
        finally:
            cursor.close()

    test_connection(conn)

    # 연결 종료
    conn.close()
    print("Database connection closed.")
